# Get all active sites for slabs
---

Analyze slabs for active sites

# Import Modules

In [1]:
import os
print(os.getcwd())
import sys

import pickle


import numpy as np
import pandas as pd

from ase import io

# # from tqdm import tqdm
from tqdm.notebook import tqdm

# # #########################################################
from proj_data import metal_atom_symbol

# #########################################################
from methods import (
    get_df_slab,
    get_structure_coord_df,
    get_df_coord,
    get_df_active_sites,
    )

# #########################################################
from local_methods import (
    mean_O_metal_coord,
    get_all_active_sites,
    get_unique_active_sites,
    get_unique_active_sites_temp,
    )

/home/raulf2012/Dropbox/01_norskov/00_git_repos/PROJ_IrOx_OER/workflow/enumerate_adsorption


# Read Data

In [2]:
# #########################################################
df_slab = get_df_slab()
df_slab = df_slab.set_index("slab_id")

# #########################################################
df_active_sites_prev = get_df_active_sites()

if df_active_sites_prev is None:
    df_active_sites_prev = pd.DataFrame()

In [5]:
df_active_sites_prev

,bulk_id,slab_id,facet,num_atoms,num_active_sites,active_sites,active_sites_unique,num_active_sites_unique
slab_id,,,,,,,,
kufiloga_64,8l919k6s7p,kufiloga_64,100,104,8,"[94, 95, 96, 97, 100, 101, 102, 103]","[94, 100]",2
nofabigo_84,8l919k6s7p,nofabigo_84,111,79,7,"[72, 73, 74, 75, 76, 77, 78]","[72, 73, 74, 75, 76, 77, 78]",7
wetokahi_33,8l919k6s7p,wetokahi_33,101,150,14,"[130, 131, 132, 135, 140, 141, 142, 143, 144, ...","[130, 132, 142, 144, 146, 148]",6
kodefivo_37,8l919k6s7p,kodefivo_37,001,63,3,"[60, 61, 62]","[60, 61]",2
lulidoka_21,8l919k6s7p,lulidoka_21,110,68,4,"[64, 65, 66, 67]","[64, 66]",2
...,...,...,...,...,...,...,...,...
runododu_73,82mwbwbgbl,runododu_73,111,101,4,"[95, 96, 99, 100]","[95, 99]",2
ripitoko_94,9573vicg7f,ripitoko_94,331,295,17,"[273, 275, 278, 279, 281, 283, 284, 285, 286, ...","[273, 278, 279, 281, 286, 287, 291, 293]",8
wefedifi_91,9573vicg7f,wefedifi_91,110,100,6,"[94, 95, 96, 97, 98, 99]","[94, 96, 98]",3


In [4]:
assert False

AssertionError: 

# Create Directories

In [ ]:
directory = "out_data"
if not os.path.exists(directory):
    os.makedirs(directory)

# TEMP

In [ ]:
# df_slab = df_slab.loc[[
#     'tiguwuti_80',
#     'sunifafa_34',
#     'lokariwu_13',
#     'fitirapa_27',
#     'nisiwuhu_70',
#     'dagaviba_06',
#     'kovudoho_21',
#     'tagilahu_40',
#     ]]

# TEMP
# df_slab = df_slab.sample(n=100)
# df_slab = df_slab.sample(n=10)
# df_slab = df_slab.sample(n=50)

In [ ]:
# TEMP
print("COMBAK REMOVE THIS")
df_slab = df_slab[df_slab.bulk_id == "8l919k6s7p"]

In [ ]:
data_dict_list = []
iterator = tqdm(df_slab.index, desc="1st loop")
for i_cnt, slab_id in enumerate(iterator):
    row_i = df_slab.loc[slab_id]
    data_dict_i = dict()

    # #####################################################
    slab = row_i.slab_final
    # slab.write("out_data/temp.cif")  # TEMP

    slab_id = row_i.name
    bulk_id = row_i.bulk_id
    facet = row_i.facet
    num_atoms = row_i.num_atoms
    # #####################################################

    if slab_id in df_active_sites_prev.index:
        run_slab_i = False
    else:
        run_slab_i = True

    # TEMP
    run_slab_i = True

    # print(run_slab_i)
    if run_slab_i:
        # #################################################
        df_coord_slab_i = get_df_coord(
            slab_id=slab_id,
            mode="slab",
            slab=slab,
            )

        # #################################################
        active_sites = get_all_active_sites(
            slab=slab,
            slab_id=slab_id,
            bulk_id=bulk_id,
            df_coord_slab_i=df_coord_slab_i,
            )

        # #################################################
        # active_sites_unique = get_unique_active_sites(
        active_sites_unique = get_unique_active_sites_temp(
            slab=slab,
            active_sites=active_sites,
            bulk_id=bulk_id,
            slab_id=slab_id,
            facet=facet,
            metal_atom_symbol=metal_atom_symbol,
            df_coord_slab_i=df_coord_slab_i,
            create_heatmap_plot=True,
            )


        # #################################################
        data_dict_i["active_sites"] = active_sites
        data_dict_i["num_active_sites"] = len(active_sites)

        data_dict_i["active_sites_unique"] = active_sites_unique
        data_dict_i["num_active_sites_unique"] = len(active_sites_unique)

        data_dict_i["slab_id"] = slab_id
        data_dict_i["bulk_id"] = bulk_id
        data_dict_i["facet"] = facet
        data_dict_i["num_atoms"] = num_atoms

        # #####################################################
        data_dict_list.append(data_dict_i)

df_active_sites = pd.DataFrame(data_dict_list)
df_active_sites = df_active_sites.set_index("slab_id", drop=False)

In [ ]:
# df_active_sites

In [ ]:
# df_active_sites.set_index

# df_active_sites = df_active_sites.set_index("slab_id", drop=False)

In [ ]:
# assert False

# Post-process active site dataframe

In [ ]:
from misc_modules.pandas_methods import reorder_df_columns

columns_list = [
    'bulk_id',
    'slab_id',
    'facet',
    'num_atoms',
    'num_active_sites',
    'active_sites',
    ]

df_active_sites = reorder_df_columns(columns_list, df_active_sites)

# Combining previous `df_active_sites` and the rows processed during current run

In [ ]:
df_active_sites = df_active_sites = pd.concat([
    df_active_sites,
    df_active_sites_prev,
    ])

# Summary of data objects

In [ ]:
print(
    "Number of active sites:",
    df_active_sites.num_active_sites.sum())
print(
    "Number of unique active sites",
    df_active_sites.num_active_sites_unique.sum())

In [ ]:
# Pickling data ###########################################
directory = "out_data"
if not os.path.exists(directory): os.makedirs(directory)
with open(os.path.join(directory, "df_active_sites.pickle"), "wb") as fle:
    pickle.dump(df_active_sites, fle)
# #########################################################

In [ ]:
print(df_active_sites.shape)

In [ ]:
assert False

In [ ]:
df_rdf_ij_dict = dict()
for i_cnt, row_i in df_active_sites.iterrows():
    file_name_i = row_i.bulk_id + "__" + row_i.facet + \
        "__" + row_i.slab_id + ".pickle"
    path_i = os.path.join(
        "out_data/df_rdf_ij", file_name_i)

    # #########################################################
    import pickle; import os
    with open(path_i, "rb") as fle:
        df_rdf_ij_i = pickle.load(fle)
    # #########################################################

    df_rdf_ij_dict[row_i.slab_id] = df_rdf_ij_i

In [ ]:
rdf_ij_list = [i for i in df_rdf_ij_i.values.flatten() if i != 0.]

In [ ]:
# num_atoms_in_slab == num_atoms_in_df

# df_coord_slab_i.shape
# slab.get_global_number_of_atoms()

In [ ]:
# df_active_sites_prev = df_active_sites_prev.set_index("slab_id")